In [ ]:
import numpy as np
from scipy import linalg
from math import sqrt

In [ ]:
def projektivno_preslikavanje_iz_baznih(A, B, C, D):
  A = np.asarray(A)
  B = np.asarray(B)
  C = np.asarray(C)
  D = np.asarray(D)

  koeficijenti = np.array([
                           [A[0], B[0], C[0]],
                           [A[1], B[1], C[1]],
                           [A[2], B[2], C[2]]
                          ])
  
  D = np.array([D[0], D[1], D[2]])

  alfa, beta, gama =  linalg.solve(koeficijenti, D)
  matrica_preslikavanja = np.array([alfa * A, beta * B, gama * C]).T

  return matrica_preslikavanja

In [ ]:
def naivni_algoritam(A, B, C, D, A1, B1, C1, D1):
  P1 = projektivno_preslikavanje_iz_baznih(A, B, C, D)
  P2 = projektivno_preslikavanje_iz_baznih(A1, B1, C1, D1)
  
  P = P2 @ linalg.inv(P1)

  return P

In [ ]:
def matrica_korespodencije(M, M1):
  M_x1, M_x2, M_x3 = M
  M1_x1, M1_x2, M1_x3 = M1

  matrica = np.array([[0, 0, 0, -M1_x3 * M_x1, -M1_x3 * M_x2, -M1_x3 * M_x3, M1_x2 * M_x1, M1_x2 * M_x2, M1_x2 * M_x3],
                      [M1_x3 * M_x1, M1_x3 * M_x2, M1_x3 * M_x3, 0, 0, 0, -M1_x1 * M_x1, -M1_x1 * M_x2, -M1_x1 * M_x3]])
  
  return matrica

In [ ]:
def DLT(*args):
  if len(args) % 2 == 1 or len(args) < 8:
    print("Lose uneti parametri!")
    return None
    
  tacke = np.array([*args])
  originali, slike = np.array_split(tacke, 2)
  
  A = np.empty(shape=[0, 9])

  for o, s in zip(originali, slike):
    A = np.vstack((A, matrica_korespodencije(o, s)))

  _, _, V = linalg.svd(A)
  P = V[-1].reshape(3, 3)

  return P

In [ ]:
def normalizacija_tacaka(*args):
  tacke = np.array([*args])
  teziste = np.array([0, 0])

  for tacka in tacke:
    tacka[0] = tacka[0] / tacka[2]
    tacka[1] = tacka[1] / tacka[2]
    teziste[0] = teziste[0] + tacka[0]
    teziste[1] = teziste[1] + tacka[1]

  teziste = teziste / len(tacke)
  
  # matrica translacije
  G = np.identity(3)
  G[0, 2] = -teziste[0]
  G[1, 2] = -teziste[1]

  # transliramo tacke
  tacke = np.delete(tacke, -1, 1)
  tacke = tacke - teziste

  # racunamo prosecno rastojanje
  d = np.sum(linalg.norm(tacke, axis=1)) / len(tacke)

  S = np.identity(3)
  S[0, 0] = S[1, 1] = sqrt(2) / d
  
  T = S @ G

  return T

In [ ]:
def normalizovani_DLT(*args):
  if len(args) % 2 == 1 or len(args) < 8:
    print("Lose uneti parametri!")
    return None
    
  tacke = np.array([*args])
  originali, slike = np.array_split(tacke, 2)

  originali = originali.astype('float64')
  slike = slike.astype('float64')

  matrica_originala = normalizacija_tacaka(*originali)
  matrica_slika = normalizacija_tacaka(*slike)

  for i, _ in enumerate(zip(originali, slike)):
    originali[i] = matrica_originala @ originali[i]
    slike[i] = matrica_slika @ slike[i]

  tacke = np.concatenate((originali, slike), axis=0)

  Pp = DLT(*tacke)
  P = linalg.inv(matrica_slika) @ Pp @ matrica_originala

  return P

**TESTIRAMO ZA 4 TACKE**

In [ ]:
print("############################# NAIVNI ALGORITAM #############################")
naivni = naivni_algoritam([1, 1, 1], [5, 2, 1], [6, 4, 1], [-1, 7, 1], [0, 0, 1], [10, 0, 1], [10, 5, 1], [0, 5, 1])
print(naivni)

############################# NAIVNI ALGORITAM #############################
[[ 0.44871795  0.14957265 -0.5982906 ]
 [-0.19230769  0.76923077 -0.57692308]
 [-0.07277526  0.07378079  0.41075918]]


In [ ]:
print("############################# DLT ALGORITAM #############################")
dlt = DLT([1, 1, 1], [5, 2, 1], [6, 4, 1], [-1, 7, 1], [0, 0, 1], [10, 0, 1], [10, 5, 1], [0, 5, 1])
print(dlt)

############################# DLT ALGORITAM #############################
[[ 0.3418787   0.11395957 -0.45583826]
 [-0.14651944  0.58607777 -0.43955833]
 [-0.05544755  0.05621367  0.31295787]]


In [ ]:
print("############################# NORMALIZOVANI DLT ALGORITAM #############################")
ndlt = normalizovani_DLT([1, 1, 1], [5, 2, 1], [6, 4, 1], [-1, 7, 1], [0, 0, 1], [10, 0, 1], [10, 5, 1], [0, 5, 1])
print(ndlt)

############################# NORMALIZOVANI DLT ALGORITAM #############################
[[ 0.49851447  0.16617149 -0.66468596]
 [-0.21364906  0.85459623 -0.64094717]
 [-0.08085151  0.08196863  0.45634321]]


In [ ]:
print("############################# NAIVNI ALGORITAM VS DLT ALGORITAM #############################")
print(f"{naivni / naivni[0, 0]}\n")
print(f"{dlt / dlt[0, 0]}\n")

############################# NAIVNI ALGORITAM VS DLT ALGORITAM #############################
[[ 1.          0.33333333 -1.33333333]
 [-0.42857143  1.71428571 -1.28571429]
 [-0.16218487  0.16442577  0.91540616]]

[[ 1.          0.33333333 -1.33333333]
 [-0.42857143  1.71428571 -1.28571429]
 [-0.16218487  0.16442577  0.91540616]]



In [ ]:
print("#############################  DLT ALGORITAM VS NORMALIZOVANI DLT #############################")
print(f"{dlt / dlt[0, 0]}\n")
print(f"{ndlt / ndlt[0, 0]}\n")

#############################  DLT ALGORITAM VS NORMALIZOVANI DLT #############################
[[ 1.          0.33333333 -1.33333333]
 [-0.42857143  1.71428571 -1.28571429]
 [-0.16218487  0.16442577  0.91540616]]

[[ 1.          0.33333333 -1.33333333]
 [-0.42857143  1.71428571 -1.28571429]
 [-0.16218487  0.16442577  0.91540616]]



 **TESTIRAMO ZA 5 TACAKA**

In [ ]:
print("############################# DLT ALGORITAM #############################")
dlt = DLT([1, 1, 1], [5, 2, 1], [6, 4, 1], [-1, 7, 1], [3, 1, 1], [0, 0, 1], [10, 0, 1], [10, 5, 1], [0, 5, 1], [3, -1, 1])
print(dlt)

############################# DLT ALGORITAM #############################
[[ 0.34349761  0.11823681 -0.48379032]
 [-0.15246704  0.58506938 -0.39503374]
 [-0.05732138  0.05556327  0.32557538]]


In [ ]:
print("############################# NORMALIZOVAN DLT ALGORITAM #############################")
ndlt = normalizovani_DLT([1, 1, 1], [5, 2, 1], [6, 4, 1], [-1, 7, 1], [3, 1, 1], [0, 0, 1], [10, 0, 1], [10, 5, 1], [0, 5, 1], [3, -1, 1])
print(ndlt)

############################# NORMALIZOVAN DLT ALGORITAM #############################
[[ 0.51947064  0.17721981 -0.72185173]
 [-0.23586042  0.89248991 -0.59143243]
 [-0.08823247  0.0846979   0.49942717]]


In [ ]:
print("#############################  DLT ALGORITAM VS NORMALIZOVANI DLT #############################")
print(f"{dlt / dlt[0, 0]}\n")
print(f"{ndlt / ndlt[0, 0]}\n")

print("Oduzimamo te dve matrice kako bi videli razliku:")
print((dlt /dlt[0, 0]) - (ndlt / ndlt[0, 0]))

#############################  DLT ALGORITAM VS NORMALIZOVANI DLT #############################
[[ 1.          0.34421437 -1.40842412]
 [-0.44386638  1.7032706  -1.15003346]
 [-0.16687564  0.16175736  0.94782431]]

[[ 1.          0.34115461 -1.389591  ]
 [-0.45403994  1.71807575 -1.13852907]
 [-0.16985074  0.16304655  0.96141557]]

Oduzimamo te dve matrice kako bi videli razliku:
[[ 0.          0.00305975 -0.01883312]
 [ 0.01017356 -0.01480515 -0.01150439]
 [ 0.0029751  -0.00128919 -0.01359127]]


In [ ]:
print("#############################  PERMUTOVANE TACKE KOD DLT ALGORITMA #############################")
print("Zamenicemo 2. i 3. tacku i uporediti ponasanje algoritma za taj slucaj\n")

r_dlt = DLT([1, 1, 1], [6, 4, 1], [5, 2, 1], [-1, 7, 1], [3, 1, 1], [0, 0, 1], [10, 5, 1], [10, 0, 1], [0, 5, 1], [3, -1, 1])

print(f"{dlt / dlt[0, 0]}\n")
print(f"{r_dlt / r_dlt[0, 0]}\n")

print("ZAKLJUCAK: algoritam se ponasa isto - nije osetljiv na ovu zamenu tacaka!")

#############################  PERMUTOVANE TACKE KOD DLT ALGORITMA #############################
Zamenicemo 2. i 3. tacku i uporediti ponasanje algoritma za taj slucaj

[[ 1.          0.34421437 -1.40842412]
 [-0.44386638  1.7032706  -1.15003346]
 [-0.16687564  0.16175736  0.94782431]]

[[ 1.          0.34421437 -1.40842412]
 [-0.44386638  1.7032706  -1.15003346]
 [-0.16687564  0.16175736  0.94782431]]

ZAKLJUCAK: algoritam se ponasa isto - nije osetljiv na ovu zamenu tacaka!


**TRANSFORMACIJA KOORDINATA - GLEDAMO PONASANJE ALGORITAMA**

In [ ]:
C1 = np.array([[0, 1, 2], [-1, 0, 3], [0, 0, 1]])
C2 = np.array([[1, -1, 5], [1, 1, -2], [0, 0, 1]])

tacke = np.array([[1, 1, 1], [5, 2, 1], [6, 4, 1], [-1, 7, 1], [3, 1, 1], [0, 0, 1], [10, 0, 1], [10, 5, 1], [0, 5, 1], [3, -1, 1]])
dlt = DLT(*tacke)

originali, slike = np.array_split(tacke, 2)

for i, (a, b) in enumerate(zip(originali, slike)):
  originali[i] = C1 @ originali[i]
  slike[i] = C2 @ slike[i]

tacke = np.concatenate((originali, slike), axis=0)

print("DLT algoritam:\n")

dlt_transformisano = DLT(*tacke)

staro = linalg.inv(C2) @ dlt_transformisano @ C1

print(f"{dlt}\n")
print(f"{dlt[0, 0] / staro[0, 0] * staro}\n")

print("ZAKLJUCAK: DLT algoritam je osetljiv na transformaciju (skaliranje) koordinata - matrice nisu iste!")

DLT algoritam:

[[ 0.34349761  0.11823681 -0.48379032]
 [-0.15246704  0.58506938 -0.39503374]
 [-0.05732138  0.05556327  0.32557538]]

[[ 0.34349761  0.11673022 -0.47437184]
 [-0.15515182  0.58975349 -0.39487542]
 [-0.05825699  0.05566205  0.33101416]]

ZAKLJUCAK: DLT algoritam je osetljiv na transformaciju (skaliranje) koordinata - matrice nisu iste!


In [ ]:
C1 = np.array([[0, 1, 2], [-1, 0, 3], [0, 0, 1]])
C2 = np.array([[1, -1, 5], [1, 1, -2], [0, 0, 1]])

tacke = np.array([[1, 1, 1], [5, 2, 1], [6, 4, 1], [-1, 7, 1], [3, 1, 1], [0, 0, 1], [10, 0, 1], [10, 5, 1], [0, 5, 1], [3, -1, 1]])
ndlt = normalizovani_DLT(*tacke)

originali, slike = np.array_split(tacke, 2)

for i, (a, b) in enumerate(zip(originali, slike)):
  originali[i] = C1 @ originali[i]
  slike[i] = C2 @ slike[i]

tacke = np.concatenate((originali, slike), axis=0)

print("Normalizovani DLT algoritam:\n")

ndlt_transformisano = normalizovani_DLT(*tacke)

staro = linalg.inv(C2) @ ndlt_transformisano @ C1

print(f"{ndlt}\n")
print(f"{ndlt[0, 0] / staro[0, 0] * staro}\n")

print("ZAKLJUCAK: normalizovani DLT algoritam nije osetljiv na transformaciju (skaliranje) koordinata - matrice su iste!")

Normalizovani DLT algoritam:

[[ 0.51947064  0.17721981 -0.72185173]
 [-0.23586042  0.89248991 -0.59143243]
 [-0.08823247  0.0846979   0.49942717]]

[[ 0.51947064  0.17721981 -0.72185173]
 [-0.23586042  0.89248991 -0.59143243]
 [-0.08823247  0.0846979   0.49942717]]

ZAKLJUCAK: normalizovani DLT algoritam nije osetljiv na transformaciju (skaliranje) koordinata - matrice su iste!


**NOVI TEST PRIMERI**

In [ ]:
naivni_novo = naivni_algoritam([2, 1, 1], [1, 2, 1], [3, 4, 1], [-1, -3, 1], [0, 1, 1], [5, 0, 1], [2, -5, 1], [-1, -1, 1])
print("Naivni algoritam:\n")
print(naivni_novo / naivni_novo[0, 0])

Naivni algoritam:

[[ 1.         -0.4893617  -1.5106383 ]
 [ 0.62765957 -0.04255319 -0.54255319]
 [ 0.5        -0.46808511  0.13829787]]


In [ ]:
dlt_novo = DLT([2, 1, 1], [1, 2, 1], [3, 4, 1], [-1, -3, 1], [0, 1, 1], [5, 0, 1], [2, -5, 1], [-1, -1, 1])
print("DLT algoritam:\n")
print(dlt_novo / dlt_novo[0, 0])

DLT algoritam:

[[ 1.         -0.4893617  -1.5106383 ]
 [ 0.62765957 -0.04255319 -0.54255319]
 [ 0.5        -0.46808511  0.13829787]]


In [ ]:
ndlt_novo = normalizovani_DLT([2, 1, 1], [1, 2, 1], [3, 4, 1], [-1, -3, 1], [0, 1, 1], [5, 0, 1], [2, -5, 1], [-1, -1, 1])
print("Normalizovani DLT algoritam:\n")
print(ndlt_novo / ndlt_novo[0, 0])

Normalizovani DLT algoritam:

[[ 1.         -0.4893617  -1.5106383 ]
 [ 0.62765957 -0.04255319 -0.54255319]
 [ 0.5        -0.46808511  0.13829787]]


In [ ]:
dlt_pet_tacaka = DLT([2, 1, 1], [1, 2, 1], [3, 4, 1], [-1, -3, 1], [-2, 5, 1], [0, 1, 1], [5, 0, 1], [2, -5, 1], [-1, -1, 1], [4, 1, 2])
print("DLT algoritam:\n")
print(dlt_pet_tacaka / dlt_pet_tacaka[0, 0])

DLT algoritam:

[[ 1.         -0.50215564 -1.45779447]
 [ 0.51618777 -0.00399712 -0.44345389]
 [ 0.46632343 -0.43192449  0.10723129]]


In [ ]:
ndlt_pet_tacaka = normalizovani_DLT([2, 1, 1], [1, 2, 1], [3, 4, 1], [-1, -3, 1], [-2, 5, 1], [0, 1, 1], [5, 0, 1], [2, -5, 1], [-1, -1, 1], [4, 1, 2])
print("Normalizovani DLT algoritam:\n")
print(ndlt_pet_tacaka / ndlt_pet_tacaka[0, 0])

Normalizovani DLT algoritam:

[[ 1.         -0.50233659 -1.45717294]
 [ 0.5161166  -0.00412331 -0.44313344]
 [ 0.46639788 -0.43189921  0.10704644]]


In [ ]:
ndlt_pet_tacaka_poslednji = normalizovani_DLT([0, -3, 1], [0, -1, 1], [4, -1, 1], [-7, -4, 1], [0, 5, 1], [3, -1, 1], [4, 4, 1], [9, 1, 1], [5, -2, 1], [7, 2, 2])
print("Normalizovani DLT algoritam (poslednji test primer):\n")
print(ndlt_pet_tacaka_poslednji / ndlt_pet_tacaka_poslednji[0, 0])

Normalizovani DLT algoritam (poslednji test primer):

[[ 1.          8.21647018 14.77346211]
 [-1.23841299  1.6150849   7.87977141]
 [-0.09224277  2.40187279  3.95457428]]
